**Django内建的test client不适合作为单元测试. 因为它测试的是整个系统: 他把你的视图作为项目环境的一个黑箱**
<br \><br \>
[这篇文章](http://tech.novapost.fr/django-unit-test-your-views-en.html)提供给开发者一个smaller, fine-grained, view-centric的测试, 来取代Django的test client

## self.client.get(): system tests for the unaware

以下是说明为什么test clien是系统测试(system test)的原因:

- 它解析URLs
- it traverses middlewares
- it traverses decorators
- it uses template context processors
- it uses template context processors
- it relies on settings

以上说的这些不是视图, 是视图需要的环境(environment).
<br \ >
这说明了, 使用test client, 你不是测试view本身, 而是整个system, view只是这个system的一部分.
<br \>
**And the environment is quite hard (and boring) to control.**

## Testing view functions

针对下面这个view: 
```Python
from django.http import Http404, HttpResponse

def hello(request, name):
    if name == u'Waldo'
        raise Http404("Where's Waldo?")
    return HttpResponse(u'Hello {name}!'.format(name=name))
```

它的测试是: 
```Python
import unittest

class HelloTestCase(unittest.TestCase):
    def test_get(self):
        """hello view actually tells 'Hello'."""
        # Setup.
        request = 'fake request'
        name = 'world'
        # Run.
        response = hello(request, name)
        # Check.
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.content, u'Hello world!')

    def test_waldo(self):
        """Cannot find Waldo to tell him 'Hello'."""
        # Setup.
        request = 'fake request'
        name = 'Waldo'
        # Run and check.
        self.assertRaises(Http404, hello, request, name)
```

挺简单的, 是吧.

## Use unittest or SimpleTestCase wherever you can

在上面的例子中, view没有用到数据库, 因此没有必要用到django.test.TransactionTestCase及其派生的类. 这样的好处是, 测试会运行得非常快(因为不用创建test database)

## Don't decorate views in place

上面的'hello'例子可能会broken, 如果在view的地方加上装饰器:
```Python
from django.http import Http404, HttpResponse
from django.contrib.auth.decorators import login_required

@login_required
def hello(request, name):
    if name == u'Waldo'
        raise Http404("Where's Waldo?")
    return HttpResponse(u'Hello {name}!'.format(name=name))
```

这样的话我们就必须要在测试的时候进行login操作, 而且我们必须要测试在登录和不登录的情况的的response的状态. 结果我们的测试会变得更长, less readable, less efficient. 而且， 万一login装饰器有bug, 或者是改变了呢? 这样'hello'的测试就可能会broke, 尽管'hello'视图本身没有改变. 同时, 有可能需要在不同的url, 使用相同的视图, 但是匹配不同的装饰器.
<br \>
**所以, don't decorate views in place.**
<br \>
Instead:
- 装饰器加到URLconfs(urls.py), 而不要直接加到view中
- 另外为装饰器写测试, 保证login_required正常工作
- 另外为URLconfs写测试, 保证login_required装饰器加到了hello上.

例子：
```Python
from django.conf.urls import patterns, url
from django.contrib.auth.decorators import login_required
from django.utils.translation import ugettext_lazy as _

from youproject.yourapp import views


people_list = login_required(views.PeopleListView.as_view())
people_detail = login_required(views.PeopleDetailView.as_view())


urlpatterns = patterns(
    '',
    url(_(r'^$'), people_list, name="people_list"),
    url(_(r'^(?P<pk>\d+)/$'), people_detail, name="people_detail"),
)
```

## Use request factories

在上面的例子中, 我们使用了假的request, 在某些场合下, 这样可能行不通, 这时候可以用django.test.RequestFactory来伪造一个请求.
<br \>
使用requeset factory, 可以获得一个request的实例, 你可以把该实例传递各view的方法, 例如dispatch().

```Python
from django.test import RequestFactory

request_factory = RequestFactory()
request = request_factory.post('/fake-path', data={'name': u'Waldo'})
```

**注意django的RequestFactory需要一个必选的参数: path. 但是在本文章的阐述中, 我们并不需要这个path, path是用于解析url, 并连接到对应的视图, 所以path参数在这里可以随意捏造一个字符串**

## Testing class-based views
fbv(函数视图)像一个黑箱: 接受一个request, 返回一个response. 没有办法测试其内部.
<br \>
cbv(类视图)有很多的属性和方法, 所以可以进行更高的粒度(fine-grained)的测试.
<br \>
The idea is: 测试每一个你写的方法和属性.
<br \>
例子:
```Python
class HelloView(TemplateView):
    def get_context_data(self, **kwargs):
        kwargs = super(HelloView, self).get_context_data(**kwargs)
        kwargs.update('name', self.kwargs.get('name'))
        return kwargs
```
And, URLconf的配置是:
```Python
view: hello = HelloView.as_view(template_name='hello.html')
URL: url(r'(?P<name>\w+)', hello)
```

### as_view() is not enough
使用as_view和RequestFactory测试cbv的方法:
```Python
import unittest
from django.test import RequestFactory

class HelloViewTestCase(unittest.TestCase):
    def test_get(self):
        """HelloView.get() sets 'name' in response context."""
        # Setup name.
        name = 'peter'
        # Setup request and view.
        request = RequestFactory().get('/fake-path')
        view = HelloView.as_view(template_name='hello.html')
        # Run.
        response = view(request, name=name)
        # Check.
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.template_name[0], 'home.html')
        self.assertEqual(response.context_data['name'], name)
```

- 上面的tes是ok的, 但是在上面的view中, 只是重写了get_context_data方法, 所以只需要测试它. 
- 我们没有办法使用as_view()获得fine-grained的测试
- 使用as_view()会返回一个函数, 而不是返回视图的实例
- 使用as_view()写测试就像根据fbv写测试一样了, 我们没有享受到根据cbv写测试的好处.
- 所以我们不要用as_view(), 而是应该关注get_context_data()

## Mimic(模仿者) as_view()

以下是取代as_view()的一个例子:
```Python
def setup_view(view, request, *args, **kwargs):
    """Mimic as_view() returned callable, but returns view instance.

    args and kwargs are the same you would pass to ``reverse()``

    """
    view.request = request
    view.args = args
    view.kwargs = kwargs
    return view
```

以下是它在测试中使用的例子:
```Python
import unittest
from django.test import RequestFactory

class HelloViewTestCase(unittest.TestCase):
    def test_context_data(self):
        """HelloView.get_context_data() sets 'name' in context."""
        # Setup name.
        name = 'django'
        # Setup request and view.
        request = RequestFactory().get('/fake-path')
        view = HelloView(template_name='hello.html')
        view = setup_view(view, request, name=name)
        # Run.
        context = view.get_context_data()
        # Check.
        self.assertEqual(context['name'], name)
```

That's all. What happened?

- 返回的是视图的实例, 因此可以测该实例的所有属性和方法
- 使用unittest是因为没有包含数据库操作

## The fairy as_view() and the ugly dispatch()

1. Using as_view() is quite elegant:
```Python
request = RequestFactory().get('/fake-path')
view = HelloView.as_view(template_name='hello.html')
response = view(request, name='bob')
```

2. Using dispatch() is ugly:

```Python
request = RequestFactory().get('/fake-path')
view = HelloView(template_name='hello.html')
view = setup_view(view, request, name='bob')
response = view.dispatch(view.request, *view.args, **view.kwargs)
```

**Got it? dispatch() receives arguments the instance already knows...**


### 上面的[例子](https://github.com/benoitbryon/django-downloadview/blob/e28951d0a14386c69e9fc00e68b4779b6f5a4791/django_downloadview/tests/views.py#L208)